In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from pydataset import data

## Attendance Data

Load the attendance.csv file and calculate an attendnace percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence. You should end up with something like this:

<code>name
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: grade, dtype: float64</code>

In [2]:
# Read in attendance.csv
df = pd.read_csv('untidy-data/attendance.csv')
df.head()

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [3]:
# Melt
df = df.rename(columns={'Unnamed: 0':'name'})
df_melt = df.melt(id_vars='name', var_name='date', value_name='status')
df_melt.head()

,name,date,status
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T


In [4]:
# Discover attendance status
# df_melt.status.unique()

# Assign new values
df_melt = df_melt.replace(to_replace={'status':{'P':1, 'A':0, 'T':0.9, 'H':0.5}})
df_melt.head()

,name,date,status
0,Sally,2018-01-01,1.0
1,Jane,2018-01-01,0.0
2,Billy,2018-01-01,0.0
3,John,2018-01-01,1.0
4,Sally,2018-01-02,0.9


In [5]:
# Calculate attendance
df_tidy = df_melt.groupby('name').status.mean()
df_tidy

name
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: status, dtype: float64

## Coffee Levels

Read the coffee_levels.csv file.
Transform the data so that each carafe is in it's own column.
Is this the best shape for the data?

In [6]:
# Read in coffee_levels
df = pd.read_csv('untidy-data/coffee_levels.csv')
df.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


In [7]:
# Pivot table of carafe
df = df.pivot_table(index='hour', columns='coffee_carafe', values='coffee_amount').reset_index()
df.head()

coffee_carafe,hour,x,y,z
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947


In [8]:
df.shape

(10, 4)

## Cake Recipes

Read the cake_recipes.csv data. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.
Tidy the data as necessary.
Which recipe, on average, is the best? recipe b
Which oven temperature, on average, produces the best results? 275
Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees

In [9]:
# Read in cake_recipes
df = pd.read_csv('untidy-data/cake_recipes.csv')
df.head()

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084


In [10]:
# Tidy data - Melt
df_melt = df.melt(id_vars='recipe:position', var_name='temp', value_name='score')
df_melt.head()

,recipe:position,temp,score
0,a:bottom,225,61.738655
1,a:top,225,51.709751
2,b:bottom,225,57.095320
3,b:top,225,82.455004
4,c:bottom,225,96.470207


In [11]:
# Tidy data - Split recipe:position
df_melt[['recipe', 'position']] = df_melt['recipe:position'].str.split(':', expand=True)
df_melt = df_melt.drop(columns='recipe:position')
df_melt.head()

,temp,score,recipe,position
0,225,61.738655,a,bottom
1,225,51.709751,a,top
2,225,57.095320,b,bottom
3,225,82.455004,b,top
4,225,96.470207,c,bottom


#### Which recipe, on average, is the best? 
recipe b 

In [12]:
df_tidy = df_melt.copy()
df_tidy.groupby('recipe').score.mean().sort_values(ascending=False).head(1)

recipe
b    76.736074
Name: score, dtype: float64

#### Which oven temperature, on average, produces the best results? 
275 

In [13]:
df_tidy.groupby('temp').score.mean().sort_values(ascending=False).head(1)

temp
275    74.886754
Name: score, dtype: float64

#### Which combination of recipe, rack position, and temperature gives the best result? 
recipe b, bottom rack, 300 degrees

In [14]:
df_tidy.sort_values(by='score', ascending=False).head(1)

,temp,score,recipe,position
26,300,99.248541,b,bottom
